# 测试集图像分类预测结果

使用训练好的图像分类模型，预测测试集的所有图像，得到预测结果表格。

## 导入工具包

In [2]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn.functional as F

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 图像预处理

In [3]:
from torchvision import transforms
from COME15KClassDataset import set_data_loader
from torchvision import datasets

# # 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
# train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
#                                       transforms.RandomHorizontalFlip(),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#                                      ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## 载入测试集（和训练代码教程相同）easy

In [6]:
# 数据集文件夹路径
dataset_name = 'test_easy_classes'
dataset_dir = '../data_class_txt/'+ dataset_name + '.txt'
# 类别名称 和 ID索引号 的映射字典
class_names_dic = {0: 'covering', 1: 'device', 2: 'domestic_animal', 3: 'mater', 4: 'person', 5: 'plant',
                       6: 'structure', 7: 'vertebrate'}

test_dataset_loader_easy = set_data_loader(dataset_attr_word="test_easy", batch_size=1, size=512, shuffle=False,
                                           transforms_compose=test_transform, dataset_dir='data/SOD-SemanticDataset')
# 载入测试集
print('测试集图像数量', len(test_dataset_loader_easy))
print('类别个数', len(class_names_dic))
print('各类别名称', list(class_names_dic.values()))
# 获得类别名称
classes = list(class_names_dic.values())
print(classes)

测试集图像数量 4600
类别个数 8
各类别名称 ['covering', 'device', 'domestic_animal', 'mater', 'person', 'plant', 'structure', 'vertebrate']
['covering', 'device', 'domestic_animal', 'mater', 'person', 'plant', 'structure', 'vertebrate']


In [4]:
## 载入测试集（和训练代码教程相同）hard

In [6]:
# 数据集文件夹路径
dataset_name = 'test_hard_classes'
model_path = '2023-10-09-01-47_max_epoch_100/'
model_name = 'retrain_COME15K_checkpoint-best-avg-0.743-Medium.pth.tar'
dataset_dir = '../data_class_txt/'+ dataset_name + '.txt'
# 类别名称 和 ID索引号 的映射字典
class_names_dic = {0: 'covering', 1: 'device', 2: 'domestic_animal', 3: 'mater', 4: 'person', 5: 'plant',
                       6: 'structure', 7: 'vertebrate'}

test_dataset_loader_easy = set_data_loader(dataset_attr_word="test_hard", batch_size=1, size=512, shuffle=False,
                                           transforms_compose=test_transform, dataset_dir='data/SOD-SemanticDataset')
# 载入测试集
print('测试集图像数量', len(test_dataset_loader_easy))
print('类别个数', len(class_names_dic))
print('各类别名称', list(class_names_dic.values()))
# 获得类别名称
classes = list(class_names_dic.values())
print(classes)

测试集图像数量 3000
类别个数 8
各类别名称 ['covering', 'device', 'domestic_animal', 'mater', 'person', 'plant', 'structure', 'vertebrate']
['covering', 'device', 'domestic_animal', 'mater', 'person', 'plant', 'structure', 'vertebrate']


## 模型路径

In [ ]:
model_path = '2023-10-09-01-47_max_epoch_100/'
model_name = 'retrain_COME15K_checkpoint-best-avg-0.743-Medium.pth.tar'

## 导入训练好的模型(导入参数)

In [1]:
from network import ShuffleNetV2_Plus
# init model
architecture = [0, 0, 3, 1, 1, 1, 0, 0, 2, 0, 2, 1, 1, 0, 2, 0, 2, 1, 3, 2]
model = ShuffleNetV2_Plus(architecture=architecture, n_class=class_names_dic.__len__(), model_size="Medium")
weight_path = model_path + model_name
trained_weight = torch.load(weight_path)
model.load_state_dict(trained_weight['state_dict'], strict=True)
model = model.eval().to(device)

NameError: name 'class_names_dic' is not defined

## 导入训练好的模型(导入模型 + 参数)

In [26]:
# model_and_weight_path = 'models/' + model_path + model_name
# model = torch.load(model_and_weight_path)
# model = model.eval().to(device)

## 表格A-测试集图像路径及标注

In [27]:
data_path = test_dataset_loader_easy.dataset.data_path

In [28]:
img_paths = test_dataset_loader_easy.dataset.images
img_lables = test_dataset_loader_easy.dataset.labels

In [29]:
df = pd.DataFrame()
df['图像路径'] = [data_path + img_name for img_name in img_paths]
df['标注类别ID'] = img_lables
df['标注类别名称'] = [class_names_dic.get(ID) for ID in img_lables]

In [30]:
df

,图像路径,标注类别ID,标注类别名称
0,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering
1,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering
2,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering
3,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering
4,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering
...,...,...,...
2995,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate
2996,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate
2997,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate
2998,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate


47_max_epoch_100## 表格B-测试集每张图像的图像分类预测结果，以及各类别置信度

In [31]:
# 记录 top-n 预测结果
n = 3

In [32]:
df_pred = pd.DataFrame()
for idx, row in tqdm(df.iterrows()):
    img_path = row['图像路径']
    img_pil = Image.open(img_path).convert('RGB')
    input_img = test_transform(img_pil).unsqueeze(0).to(device) # 预处理
    pred_logits = model(input_img) # 执行前向预测，得到所有类别的 logit 预测分数
    pred_softmax = F.softmax(pred_logits, dim=1) # 对 logit 分数做 softmax 运算

    pred_dict = {}

    top_n = torch.topk(pred_softmax, n) # 取置信度最大的 n 个结果
    pred_ids = top_n[1].cpu().detach().numpy().squeeze() # 解析出类别
    
    # top-n 预测结果
    for i in range(1, n+1):
        pred_dict['top-{}-预测ID'.format(i)] = pred_ids[i-1]
        pred_dict['top-{}-预测名称'.format(i)] = class_names_dic.get(pred_ids[i-1])
    pred_dict['top-n预测正确'] = row['标注类别ID'] in pred_ids
    # 每个类别的预测置信度
    for idx, each in enumerate(classes):
        pred_dict['{}-预测置信度'.format(each)] = pred_softmax[0][idx].cpu().detach().numpy()
        
    df_pred = df_pred._append(pred_dict, ignore_index=True)

3000it [00:39, 75.48it/s]


In [33]:
df_pred

,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,covering-预测置信度,device-预测置信度,domestic_animal-预测置信度,mater-预测置信度,person-预测置信度,plant-预测置信度,structure-预测置信度,vertebrate-预测置信度
0,0,covering,4,person,3,mater,True,0.80111164,0.0068586385,0.0025501177,0.015600368,0.15835266,0.0062205354,0.005664707,0.0036413413
1,6,structure,3,mater,0,covering,True,0.11338254,0.027639503,0.0060228454,0.2592547,0.044438392,0.00682551,0.5383202,0.0041163424
2,0,covering,4,person,2,domestic_animal,True,0.60094273,0.008279729,0.050587278,0.0327518,0.27734488,0.0051054903,0.013125971,0.011862167
3,4,person,3,mater,1,device,False,0.017681606,0.046715472,0.005153026,0.08719274,0.81334174,0.0037874614,0.019615391,0.0065125967
4,0,covering,3,mater,4,person,True,0.6815421,0.039884984,0.0126474975,0.15696844,0.054005932,0.019468576,0.012043632,0.023438899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,7,vertebrate,1,device,6,structure,True,0.018748438,0.050075125,0.029230088,0.023155669,0.016222306,0.0074414066,0.02983271,0.82529426
2996,7,vertebrate,2,domestic_animal,3,mater,True,0.007847314,0.00799435,0.03485909,0.01713725,0.0052436725,0.010045065,0.011848277,0.905025
2997,4,person,2,domestic_animal,0,covering,False,0.16148382,0.028713375,0.18258692,0.11862688,0.27012292,0.04483563,0.03971726,0.15391326
2998,3,mater,7,vertebrate,5,plant,True,0.011064374,0.021489361,0.0102615645,0.72749645,0.012829391,0.033281658,0.020165678,0.1634115


## 拼接AB两张表格

In [34]:
df = pd.concat([df, df_pred], axis=1)

In [35]:
df

,图像路径,标注类别ID,标注类别名称,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,covering-预测置信度,device-预测置信度,domestic_animal-预测置信度,mater-预测置信度,person-预测置信度,plant-预测置信度,structure-预测置信度,vertebrate-预测置信度
0,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering,0,covering,4,person,3,mater,True,0.80111164,0.0068586385,0.0025501177,0.015600368,0.15835266,0.0062205354,0.005664707,0.0036413413
1,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering,6,structure,3,mater,0,covering,True,0.11338254,0.027639503,0.0060228454,0.2592547,0.044438392,0.00682551,0.5383202,0.0041163424
2,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering,0,covering,4,person,2,domestic_animal,True,0.60094273,0.008279729,0.050587278,0.0327518,0.27734488,0.0051054903,0.013125971,0.011862167
3,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering,4,person,3,mater,1,device,False,0.017681606,0.046715472,0.005153026,0.08719274,0.81334174,0.0037874614,0.019615391,0.0065125967
4,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering,0,covering,3,mater,4,person,True,0.6815421,0.039884984,0.0126474975,0.15696844,0.054005932,0.019468576,0.012043632,0.023438899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate,7,vertebrate,1,device,6,structure,True,0.018748438,0.050075125,0.029230088,0.023155669,0.016222306,0.0074414066,0.02983271,0.82529426
2996,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate,7,vertebrate,2,domestic_animal,3,mater,True,0.007847314,0.00799435,0.03485909,0.01713725,0.0052436725,0.010045065,0.011848277,0.905025
2997,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate,4,person,2,domestic_animal,0,covering,False,0.16148382,0.028713375,0.18258692,0.11862688,0.27012292,0.04483563,0.03971726,0.15391326
2998,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate,3,mater,7,vertebrate,5,plant,True,0.011064374,0.021489361,0.0102615645,0.72749645,0.012829391,0.033281658,0.020165678,0.1634115


## 导出完整表格

In [36]:
df.to_csv('./models/' + model_path + dataset_name +'-测试集预测结果.csv', index=False)